In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# To create mock images
from jf_wep.donutStamp import DonutStamp
from jf_wep.imageMapper import ImageMapper
from scipy.ndimage import gaussian_filter

# To estimate Zernikes
from jf_wep.wfEstimator import WfEstimator

### Create some mock data

Generate some random Zernikes

In [ ]:
# Set some random Zernikes
rng = np.random.default_rng(1243)
zkTrue = rng.normal(0, 5e-6 / np.arange(1, 26) ** 2, size=25)

fig, ax = plt.subplots()
ax.plot(np.arange(4, 29), zkTrue)
ax.set(
    xlabel="Zernike Noll index",
    ylabel="Amplitude (meters)",
    title="True Zernikes",
)
plt.show()


Forward model a pair of donuts

In [ ]:
# Create the ImageMapper to forward model images
mapper = ImageMapper()

# Save mapper info for use when estimating Zernikes
wfEstConfig = {
    "instConfig": mapper.instrument,
    "algoConfig": {"addIntrinsic": mapper.addIntrinsic},
}


pupil = mapper.instrument.createPupilMask(180).astype(float)

intraStamp = mapper.pupilToImage(
    DonutStamp(
        120 * pupil,
        (0, 0),
        "intra",
    ),
    zkTrue,
)
intraStamp.image = gaussian_filter(intraStamp.image, 5) + rng.normal(
    scale=10, size=pupil.shape
)

extraStamp = mapper.pupilToImage(
    DonutStamp(
        60 * pupil,
        (0, 0),
        "extra",
    ),
    zkTrue,
)
extraStamp.image = gaussian_filter(extraStamp.image, 5) + rng.normal(
    scale=15, size=pupil.shape
)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5, 2), dpi=180)

ax1.imshow(intraStamp.image, origin="lower")
ax1.set(title="Intra")

ax2.imshow(extraStamp.image, origin="lower")
ax2.set(title="Extra")

plt.show()


### Estimate Zernikes with TIE (exp solver)

In [ ]:
zkTieExp = WfEstimator(**wfEstConfig).estimateWf(intraStamp, extraStamp)

In [ ]:
fig, ax = plt.subplots()

ax.plot(4 + np.arange(len(zk)), zk, label="True")
ax.plot(4 + np.arange(len(zkTieExp)), zkTieExp, label="TIE (exp)", ls="--")
ax.legend()

ax.set(xlabel="Zernike Noll index", ylabel="Amplitude (meters)")
plt.show()